In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import *
from tensorflow.keras.models import *

2023-04-03 11:02:41.894210: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-03 11:02:42.020505: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-03 11:02:42.047509: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a suppor

# DeConvNet. 
- 전치컨볼루션(Transposed Convolution)을 DeConvolution이라고도 부름

In [16]:
def DeConvNet(input_shape, num_classes):
    # 입력 이미지를 받기 위한 placeholder
    inputs = Input(shape=input_shape)

    #### Encoder ####
    
    # Block 1
    x = Conv2D(64, kernel_size=(3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(64, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x1 = MaxPooling2D(pool_size=(2, 2))(x)

    # Block 2
    x = Conv2D(128, kernel_size=(3, 3), padding='same')(x1)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(128, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x2 = MaxPooling2D(pool_size=(2, 2))(x)

    # Block 3
    x = Conv2D(256, kernel_size=(3, 3), padding='same')(x2)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(256, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x3 = MaxPooling2D(pool_size=(2, 2))(x)

    # Block 4
    x = Conv2D(512, kernel_size=(3, 3), padding='same')(x3)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x4 = MaxPooling2D(pool_size=(2, 2))(x)

    # Block 5
    x = Conv2D(512, kernel_size=(3, 3), padding='same')(x4)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(512, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x5 = MaxPooling2D(pool_size=(2, 2))(x)

    
    
    #### Decoder ####

    # Block 1
    x = Conv2DTranspose(512, kernel_size=(3, 3), padding='same')(x5)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(512, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D(size=(2, 2))(x)

    # Block 2
    x = Conv2DTranspose(256, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(256, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D(size=(2, 2))(x)

    # Block 3
    x = Conv2DTranspose(128, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(128, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D(size=(2, 2))(x)

    # Block 4
    x = Conv2DTranspose(64, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2DTranspose(64, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Output
    x = Conv2D(num_classes, kernel_size=(1, 1), padding='valid')(x)
    output = Activation('softmax')(x)

    # 모델 구성
    model = Model(inputs=inputs, outputs=output)

    return model


In [17]:
input_shape = (224, 224, 3); num_classes = 10

# 모델 생성
model = DeConvNet(input_shape, num_classes)

model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_561 (Conv2D)         (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization_480 (Ba  (None, 224, 224, 64)     256       
 tchNormalization)                                               
                                                                 
 activation_301 (Activation)  (None, 224, 224, 64)     0         
                                                                 
 conv2d_562 (Conv2D)         (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_481 (Ba  (None, 224, 224, 64)     256       
 tchNormalization)                                         